In [1]:
import pandas as pd
from plotly.subplots import make_subplots
import pycountry_convert as pc

# 1. Llegim el fitxer en format Excel

In [2]:
ruta_document = 'migracio.xlsx'

df = pd.read_excel(ruta_document, sheet_name='Table 1', header=10)

# 2. Canviem nomes de columnes i afegim el continent al qual pertanyen

In [20]:
df = df.rename(columns={'Region, development group, country or area of origin': 'Origin',
                        'Region, development group, country or area of destination': 'Destination', 
                        'Location code of destination': 'cod_dest', 
                        'Location code of origin': 'cod_orig'})

df['Origin'] = df['Origin'].str.replace('*', '')
df['Origin'] = df['Origin'].str.strip()


df['Destination'] = df['Destination'].str.replace('*', '')
df['Destination'] = df['Destination'].str.strip()


paisos = df[(df['cod_orig'] < 900) & (df['cod_dest'] < 900)]


codis_desitjats = [910, 911, 912, 913, 914, 923, 924, 925, 926, 5500, 906, 920, 5501, 922, 915, 916, 931, 905, 927, 928, 954, 957]

zones = df[(df['cod_orig'].isin(codis_desitjats)) & (df['cod_dest'].isin(codis_desitjats))]


def country_to_continent(country_code):
    try:
        country_alpha2 = pc.country_name_to_country_alpha2(country_code)
        continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)
        return continent_name
    except:
        return None


paisos['Continent_origin'] = df['Origin'].apply(country_to_continent)
paisos['Continent_dest'] = df['Destination'].apply(country_to_continent)

# apliquem manualment els continents que no s'han aplicat amb la funció 

correspondencia = {
    'China, Hong Kong SAR': 'Asia',
    "Dem. People's Republic of Korea": 'Asia',
    'Republic of Korea': 'Asia',
    'State of Palestine': 'Asia',
    'Western Sahara': 'Africa',
    'Iran (Islamic Republic of)': 'Asia',
    'Venezuela (Bolivarian Republic of)': 'South America',
    'Bolivia (Plurinational State of)': 'South America',
    'Saint Helena': 'Africa',
    'Timor-Leste': 'Asia',
    'Micronesia (Fed. States of)': 'Oceania',
    'China, Macao SAR': 'Asia',
    'Holy See': 'Europe',
    'Wallis and Futuna Islands': 'Oceania',
    'Channel Islands': 'Europe',
    'Sint Maarten (Dutch part)': 'North America'
}

paisos['Continent_origin'] = paisos['Origin'].map(correspondencia).fillna(paisos['Continent_origin'])
paisos['Continent_dest'] = paisos['Destination'].map(correspondencia).fillna(paisos['Continent_dest'])



paisos.to_csv('paisos2.csv', index=False)




/var/folders/d9/hpj0n4ds4lb3546xfwjxjmcm0000gq/T/ipykernel_82787/1668529684.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Origin'] = df['Origin'].str.replace('*', '')
/var/folders/d9/hpj0n4ds4lb3546xfwjxjmcm0000gq/T/ipykernel_82787/1668529684.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Destination'] = df['Destination'].str.replace('*', '')
/var/folders/d9/hpj0n4ds4lb3546xfwjxjmcm0000gq/T/ipykernel_82787/1668529684.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

array([], dtype=object)

# 3. Afegim Longitud i Latitud dels paisos tant d'origen com de destinació i creem un arxiu GeoJson amb les línies de cada combinació de paisos

In [21]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import LineString

paisos = pd.read_csv('paisos2.csv')
df_longitude_latitude = pd.read_csv('longitude-latitude.csv')

# afegim manualment paisos que no consten al dataset

nous_països = pd.DataFrame({
    'Country': ['Democratic Republic of the Congo', 'United Republic of Tanzania',
                'China, Hong Kong SAR', 'Cabo Verde', "Dem. People's Republic of Korea",
                'Republic of Korea', 'State of Palestine', 'Republic of Moldova',
                'Micronesia (Fed. States of)', 'China, Macao SAR',
                'Wallis and Futuna Islands', 'Channel Islands'],
    'Latitude': [-4.0383, -6.369028, 22.3193, 16.5388, 40.3399, 35.9078, 31.9522, 47.4116, 6.8874, 22.1987, -13.7685, 49.3723],
    'Longitude': [21.7587, 34.888822, 114.1694, -23.0418, 127.5101, 127.7669, 35.2332, 28.3699, 158.2150, 113.5439, -177.1561, -2.3644]
})

# Afegim les noves files al DataFrame original

df_longitude_latitude = pd.concat([df_longitude_latitude, nous_països], ignore_index=True)


paisos = pd.merge(paisos, df_longitude_latitude, left_on='Origin', right_on='Country', how='left')

paisos = paisos.rename(columns={'Longitude': 'Orig_longitude', 'Latitude': 'Orig_latitude'})

paisos = pd.merge(paisos, df_longitude_latitude, left_on='Destination', right_on='Country', how='left')

paisos = paisos.rename(columns={'Longitude': 'Dest_longitude', 'Latitude': 'Dest_latitude'})

paisos = paisos.drop(['Country_x', 'Country_y'], axis=1, errors='ignore')

columns_to_keep = ['Destination', 'Origin', '1990', '1995', '2000', '2005', '2010', '2015', '2020',
                   '1990.1', '1995.1', '2000.1', '2005.1', '2010.1', '2015.1', '2020.1',
                   '1990.2', '1995.2', '2000.2', '2005.2', '2010.2', '2015.2', '2020.2',
                   'Continent_origin', 'Continent_dest', 
                   'Orig_latitude', 'Orig_longitude', 'Dest_latitude', 'Dest_longitude']

paisos = paisos.loc[:, columns_to_keep]

paisos.replace({'Origin': {'Russian Federation': 'Russia'},
            'Destination': {'Russian Federation': 'Russia'}}, inplace=True)


paisos['Line'] = [LineString([(orig_lon, orig_lat), (dest_lon, dest_lat)]) for orig_lon, orig_lat, dest_lon, dest_lat in zip(paisos['Orig_longitude'], paisos['Orig_latitude'], paisos['Dest_longitude'], paisos['Dest_latitude'])]

# Creem un GeoDataFrame a partir del DataFrame amb les línies com a geometries

gdf_lines = gpd.GeoDataFrame(paisos, geometry='Line')

# Guardem el GeoDataFrame com a GeoJSON
gdf_lines.to_file("output.geojson", driver="GeoJSON")


# Crear les columnes de tipus geometry per a les coordenades d'origen i destinació
paisos['Coord_origin'] = paisos.apply(lambda row: Point(row['Orig_longitude'], row['Orig_latitude']), axis=1)
paisos['Coord_dest'] = paisos.apply(lambda row: Point(row['Dest_longitude'], row['Dest_latitude']), axis=1)


paisos = paisos.dropna(subset=['Coord_origin'])
paisos = paisos.dropna(subset=['Coord_dest'])



paisos.to_csv('paisos3.csv', index=False)




# 4. Creem un dataset amb els anys per files i els continents d'origen per columnes

In [22]:
# Selecciona només les columnes d'interès (Country i les columnes d'any)

columns_of_interest = ['Origin', '1990', '1995', '2000', '2005', '2010', '2015', '2020']
df2 = paisos[columns_of_interest]

df_melted = pd.melt(df2, id_vars=['Origin'], var_name='Year', value_name='Value')

df_grouped = df_melted.groupby(['Origin', 'Year']).sum().reset_index()

df_pivoted = df_grouped.pivot(index='Year', columns='Origin', values='Value').reset_index()

df_pivoted = df_pivoted.rename_axis(None, axis=1).reset_index()

df_pivoted.dtypes


index                         int64
Year                         object
Afghanistan                   int64
Albania                       int64
Algeria                       int64
                              ...  
Wallis and Futuna Islands     int64
Western Sahara                int64
Yemen                         int64
Zambia                        int64
Zimbabwe                      int64
Length: 234, dtype: object

# 5. Creem un dataset que correspon al balanç de migrants de l'any 2020 per pais i el percentatge d'homes i dones

In [23]:
migrants_marxen = paisos.groupby('Origin')[['2020', '2020.1', '2020.2']].sum().reset_index()
migrants_marxen = migrants_marxen.rename(columns={'2020': 'Migrants que marxen', '2020.1': 'Homes que marxen', '2020.2': 'Dones que marxen'})

# Agrupem per país de destinació i suma la quantitat de migrants per cada columna
migrants_arriben = paisos.groupby('Destination')[['2020', '2020.1', '2020.2']].sum().reset_index()
migrants_arriben = migrants_arriben.rename(columns={'2020': 'Migrants que arriben', '2020.1': 'Homes que arriben', '2020.2': 'Dones que arriben'})
resultat = pd.merge(migrants_marxen, migrants_arriben, how='outer', left_on='Origin', right_on='Destination')

resultat = resultat.fillna(0)

# Calculem la diferència entre migrants que marxen i migrants que arriben per cada columna
resultat['Balanç Total'] = resultat['Migrants que arriben'] - resultat['Migrants que marxen']
resultat['% Homes Arriben'] = resultat['Homes que arriben']*100/resultat['Migrants que arriben']
resultat['% Dones Arriben'] = resultat['Dones que arriben']*100/resultat['Migrants que arriben']
resultat['% Homes Marxen'] = resultat['Homes que marxen']*100/resultat['Migrants que marxen']
resultat['% Dones Marxen'] = resultat['Dones que marxen']*100/resultat['Migrants que marxen']

resultat = resultat.drop(['Destination'], axis=1)

resultat.to_csv('paisos4.csv', index=False)



# 6. Creem un dataset agrupat per país d'origen

In [26]:
columnes_interes = ['Origin', '1990', '1995', '2000', '2005', '2010', '2015', '2020', 'Continent_origin']

resultat_groupby = paisos[columnes_interes].groupby(['Origin', 'Continent_origin']).sum().reset_index()

resultat_groupby.to_csv('paisos6.csv', index=False)



# 7. Creem un dataset que correspon a combinacions de paisos que al 2020 tinguin més de 100.000 migrants

In [24]:
df_seleccionat2 = paisos[['Origin', 'Destination', '2020', 'Continent_origin', 'Continent_dest']]

df_filtrat2 = df_seleccionat2[df_seleccionat2['2020'] > 100000]
df_filtrat2.to_csv('paisos5.csv', index=False)
